In [1]:
pip install git+https://github.com/pulakeshpradhan/geepack.git


  Cloning https://github.com/pulakeshpradhan/geepack.git to c:\users\pulak\appdata\local\temp\pip-req-build-az3m1ht2
  Resolved https://github.com/pulakeshpradhan/geepack.git to commit 9c58942fd0ec4190d3d2ec6341e070c877f60eca
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/pulakeshpradhan/geepack.git 'C:\Users\pulak\AppData\Local\Temp\pip-req-build-az3m1ht2'


In [2]:
from gee_export_utils import export_image_to_drive

ModuleNotFoundError: No module named 'gee_export_utils'

In [ ]:
import ee
import geemap


# Initialize Earth Engine
ee.Initialize()

# Define an image and a region
image = ee.Image('USGS/SRTMGL1_003')
region = ee.Geometry.Rectangle([-122.092, 37.42, -122.08, 37.43])

# Export the image to Google Drive
export_image_to_drive(image, region, scale=30, description='srtm_export')

# Download the image as a single GeoTIFF
# download_image_single(image, region, scale=30, output_dir='./downloads')

# Download the image as tiles
# download_image_tiles(image, region, scale=30, output_dir='./tiles', rows=2, cols=2)